In [1]:
import pandas as pd
import blackjack_utils.game_config as gc
import blackjack_utils.shoe as shoe
import blackjack_utils.card as card
import random
import blackjack_utils.deck as deck
from  blackjack_utils.utils import simulate_hand, build_combos, determine_best_action

In [2]:
ev_action_table = pd.read_csv('action_expected_values_fuller.csv')
ev_action_table['player_total'] = ev_action_table['player_total'].astype(str)
ev_action_table['dealer_card_up'] = ev_action_table['dealer_card_up'].astype(str)

In [3]:
combos = build_combos()

In [ ]:
import random
# now do it for 18-12
for player_amt in ['11', '10', '9', '8', '7', '6', '5', '4']:
    ev_action_table_stand = ev_action_table.copy()
    ev_action_table_stand['best_action'] = 'stand'
    ev_action_table_stand['player_total'] = player_amt
    ev_action_table_stand.drop_duplicates(subset=['player_total', 'dealer_card_up'], inplace=True)
    ev_action_table_double = ev_action_table_stand.copy()
    ev_action_table_stand = pd.concat([ev_action_table, ev_action_table_stand], ignore_index=True)
    ev_action_table_double['best_action'] = 'double'
    ev_action_table_hit = ev_action_table_double.copy()
    ev_action_table_double = pd.concat([ev_action_table_double, ev_action_table], ignore_index=True)
    ev_action_table_hit['best_action'] = 'hit'
    ev_action_table_hit = pd.concat([ev_action_table_hit, ev_action_table], ignore_index=True)
    combos_for_19 = combos[player_amt]
    game_config = gc.GameConfig(6, True, True, True, 1.5)
    for dealer_card_rank in list(range(9)) + [12]:
        hit_total = 0
        double_total = 0
        stand_total = 0
        for i in range(40000):
            if i % 1000 == 0:
                print(f"dealer_card_rank: {dealer_card_rank}, i: {i}, player_total: {player_amt}")
            deck = shoe.Shoe(6)
            deck.shuffle()
            player_cards = list(random.choice(combos_for_19))
            dealer_card_up = card.Card().from_ints(dealer_card_rank, 0)
            deck.remove(player_cards[0])
            deck.remove(player_cards[1])
            deck.remove(dealer_card_up)
            hit_total += simulate_hand(game_config, player_cards, dealer_card_up, deck, ev_action_table_hit)
            double_total += simulate_hand(game_config, player_cards, dealer_card_up, deck, ev_action_table_double)
            stand_total += simulate_hand(game_config, player_cards, dealer_card_up, deck, ev_action_table_stand)
        ev_action_table = pd.concat([ev_action_table, pd.DataFrame({'player_total': [player_amt], 'dealer_card_up': [dealer_card_up.get_card_value()], 'double': [double_total/40000], 'hit': [hit_total/40000], 'stand': [stand_total/40000]})], ignore_index=True)
        ev_action_table['best_action'] = ev_action_table.apply(determine_best_action, axis=1)

In [ ]:
ev_action_table.to_csv('action_expected_values_fuller_pt2.csv')